<a href="https://colab.research.google.com/github/soyoonjeong/PyTorch_Tutorial/blob/main/Build_the_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

In [19]:
device =  "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device") # 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 함, 가능하지 않으면 CPU 계속 사용

Using cuda device


## 클래스 정의하기
신경망 모델을 nn.Module의 하위 클래스로 정의하고 \_\_init__에서 신경망 계층들을 초기화, forward에서 입력 데이터에 대한 연산들을 구현

In [24]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten() # dim = 0의 미니배치 차원은 유지한 채로 2d 이미지를 연속된 배열로 변환
    self.linear_relu_stack = nn.Sequential( # 3층 신경망, nn.Sequential은 순서를 갖는 모듈의 컨테이너
        nn.Linear(28*28, 512), # 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용
        nn.ReLU(), # 활성화 함수 : 모델의 입력과 출력 사이에 비선형성 도입
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [25]:
model = NeuralNetwork().to(device = device) # 신경망 모델 인스턴스 생성하고 이를 device로 이동
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [26]:
X = torch.randn((1, 28, 28), device = device)
logits = model(X) # 모델을 사용하기 위해 입력 데이터 전달, 일부 백그라운드 연산들과 함께 모델의 forward를 실행 (직접 forward() 호출 X)
print(logits)
pred_probab = nn.Softmax(dim = 1)(logits) # 예측 확률
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

tensor([[ 0.1333, -0.0055, -0.1402, -0.1327, -0.0817,  0.0821, -0.1909,  0.0133,
         -0.0660, -0.1304]], device='cuda:0', grad_fn=<AddmmBackward0>)
Predicted class : tensor([0], device='cuda:0')


## 모델 매개변수

In [29]:
print(f"Model Structure : {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model Structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0176,  0.0031,  0.0060,  ..., -0.0195, -0.0290, -0.0353],
        [-0.0020,  0.0073,  0.0272,  ..., -0.0162, -0.0284,  0.0053]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0238,  0.0038], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0244, -0.0282,  0.0178,  ...,  0.0329, -0.0338,  0.0159],
        [-0.0030,  0.0359, -0.0231,  ...,  0.0195,  0.0251, -0.0292]],
       device='cuda:0', grad